In [64]:
# Importing necessary libs
import pandas as pd
import numpy as np
import os
import plotly.express as px

In [65]:
import os
import pandas as pd

PATH = 'data/'


def merge_datasets():
    """
    Function to merge all the datasets in the specified directory and create a single dataset
    with an additional 'spending_type' column to identify income and expenses.
    """
    df_income = [] 
    df_expense = []
    for file in os.listdir(PATH):
        if file.endswith('.xlsx') or file.endswith('.csv'):  # Corrected the condition
            # Data with income
            df_income_temp = pd.read_excel(f'{PATH}{file}', skiprows=1, sheet_name='Income')
            df_income_temp['spending_type'] = 'Income'  # identifier column
            df_income.append(df_income_temp)

            # Data with expense
            df_expense_temp = pd.read_excel(f'{PATH}{file}', skiprows=1, sheet_name='Expenses')
            df_expense_temp['spending_type'] = 'Expense'  # identifier column
            df_expense.append(df_expense_temp)

    # concatenate income and expense data into a single DataFrame
    df_income = pd.concat(df_income, ignore_index=True)
    df_expense = pd.concat(df_expense, ignore_index=True)

    # merge income and expense data
    df_combined = pd.concat([df_income, df_expense], ignore_index=True)
    
    # Cleaning the data
    df_combined.sort_values(by='Date and time', inplace=True)
    # remove columns
    df_combined.drop(['Amount in account currency', 'Account currency','Transaction amount in transaction currency', 'Transaction currency', 'Tags', 'Account','Default currency', 'Comment'], axis=1, inplace=True) # can include if needed

    return df_combined




In [66]:
df = merge_datasets()
df_expense= df[df['spending_type'] == 'Expense']
df_income = df[df['spending_type'] == 'Income']

In [67]:
total_expense = df_expense['Amount in default currency'].sum()
total_income = df_income['Amount in default currency'].sum()

print(f"Total income: {total_income}")
print(f"Total expense: {total_expense}")

Total income: 45533.14
Total expense: 41572.86


In [68]:
# plotting income and expense
fig = px.box(df_expense, 
             x='spending_type', 
             y='Amount in default currency', 
             color='spending_type', 
             width=800, 
             height=800 ,
             title='Total Income and Expense',
             labels=
             {'spending_type':'Type of spending', 'Amount in default currency':'Amount'}
            )
fig.show()

In [69]:

# grouped by category for expense
df_expense_grouped = df_expense.groupby('Category', as_index=False)['Amount in default currency'].sum()

# plotting expenses by category
fig = px.bar(df_expense_grouped, 
             x='Category', 
             y='Amount in default currency', 
             color='Category', 
             title='Total Expense by Category',
             labels={'Category': 'Category', 'Amount in default currency': 'Amount in CAD'},
             width=1200,
             height=800,
            )

fig.show()


In [70]:

# grouped by category for income
df_income_grouped = df_income.groupby('Category', as_index=False)['Amount in default currency'].sum()

# plotting expenses by category
fig = px.bar(df_income_grouped, 
             x='Category', 
             y='Amount in default currency', 
             color='Category', 
             title='Total Income by Category',
             labels={'Category': 'Category', 'Amount in default currency': 'Amount in CAD'},
             width=1200,
             height=800,
            )

fig.show()


In [71]:
display(df_income_grouped, df_expense_grouped)

,Category,Amount in default currency
0,Balancing,321.52
1,Cashback,26.00
2,Other,1351.00
3,Paycheck,12678.92
4,TAX RETURNS,1590.00
5,Transfer,29565.70


,Category,Amount in default currency
0,Balancing,171.93
1,Cafe,4678.76
2,Education,19795.87
3,Fashion,428.78
4,Gadgets,85.71
5,Gifts,627.72
6,Groceries,2340.62
7,Haircut,65.55
8,Home,10700.00
9,Laptop repair,135.60


In [72]:
# grouped by date and time for income and expense
df_grouped = df.groupby(['Date and time', 'spending_type'], as_index=False)['Amount in default currency'].sum()

fig = px.line(df_grouped, 
              x='Date and time', 
              y='Amount in default currency', 
              color='spending_type',  # Different colors for Income and Expense
              title='Time Series of Income and Expense',
              labels={'spending_type': 'Type', 'Amount in default currency': 'Amount in CAD'},
              width=1900,
              height=900,
              
             )

fig.show()
